In [2]:
import json 
import tensorflow as tf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from tensorflow.keras import backend as K
from keras.models import Model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Embedding, Conv1D, Conv2D, MaxPooling1D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers import SpatialDropout1D, concatenate, BatchNormalization
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.callbacks import Callback
from keras.optimizers import Adam
from keras.utils import plot_model

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Importer les Train et test pour les titres 

In [3]:
data_dir = "data/title_only"
feature = "title"
label = 'label'
x_train = pd.read_csv(f"{data_dir}/train.csv")[feature]
y_train = pd.read_csv(f"{data_dir}/train.csv")[label]

x_test = pd.read_csv(f"{data_dir}/test.csv")[feature]
y_test = pd.read_csv(f"{data_dir}/test.csv")[label]

x_train.shape, y_train.shape, x_test.shape, y_test.shape,

((8000,), (8000,), (2000,), (2000,))

In [4]:
#manipulation de y:catégorie
def create_vocab(dt): 
  to_id = {'<PAD>': 0, '<UNK>':1}

  for sent in dt:
    for w in sent: 
      if w not in to_id.keys():
        to_id[w] = len(to_id)

  from_id = {v: k for k, v in to_id.items()}

  vocab = len(to_id.keys())

  return to_id, from_id, vocab

def preprocess_Y(Y, cat_to_id): 
  res = []
  for ex in Y: 
    if ex not in cat_to_id.keys():
      res.append(cat_to_id['<UNK>'])
    else:
      res.append(cat_to_id[ex])
  return np.array(res)

cat_to_id, cat_from_id, cat_vocab = create_vocab([y_train])
y_train_id = preprocess_Y(y_train, cat_to_id)
y_test_id = preprocess_Y(y_test, cat_to_id)

#Manipulation des titres 
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train)

x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq =  tokenizer.texts_to_sequences(x_test)

x_train_seq = tf.keras.preprocessing.sequence.pad_sequences(x_train_seq, maxlen = 40, truncating='post')
x_test_seq = tf.keras.preprocessing.sequence.pad_sequences(x_test_seq, maxlen = 40, truncating='post')

In [4]:
#une fonction pour visualiser accuracy et loss
def plot_history(history, metric="acc"):
    # summarize history for accuracy
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.title(f'model {metric}')
    plt.ylabel(metric)
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [10]:
tokenizer.word_index

{'of': 1,
 'and': 2,
 'in': 3,
 'for': 4,
 'the': 5,
 'a': 6,
 'using': 7,
 'to': 8,
 'with': 9,
 'neural': 10,
 'based': 11,
 'learning': 12,
 'on': 13,
 'analysis': 14,
 'robotic': 15,
 'network': 16,
 'by': 17,
 'networks': 18,
 'an': 19,
 'assisted': 20,
 'artificial': 21,
 'from': 22,
 'robot': 23,
 'prediction': 24,
 'machine': 25,
 'system': 26,
 'cancer': 27,
 'deep': 28,
 'data': 29,
 'laparoscopic': 30,
 'classification': 31,
 'surgery': 32,
 'model': 33,
 'approach': 34,
 'detection': 35,
 'protein': 36,
 'study': 37,
 'prostatectomy': 38,
 'human': 39,
 'fuzzy': 40,
 'radical': 41,
 'gene': 42,
 'computer': 43,
 'patients': 44,
 'identification': 45,
 'disease': 46,
 'automated': 47,
 'support': 48,
 'method': 49,
 'images': 50,
 'application': 51,
 'brain': 52,
 'diagnosis': 53,
 'image': 54,
 'control': 55,
 'novel': 56,
 'predicting': 57,
 'imaging': 58,
 'models': 59,
 'during': 60,
 'clinical': 61,
 'comparison': 62,
 'new': 63,
 'development': 64,
 'expression': 65,
 

In [5]:
# Fonction permettant de charger un embedding 

import numpy as np
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def load_glove_embeddings(fp, embedding_dim, include_empty_char=True):
    """
    Loads pre-trained word embeddings (GloVe embeddings)
        Inputs: - fp: filepath of pre-trained glove embeddings
                - embedding_dim: dimension of each vector embedding
                - generate_matrix: whether to generate an embedding matrix
        Outputs:
                - word2coefs: Dictionary. Word to its corresponding coefficients
                - word2index: Dictionary. Word to word-index
                - embedding_matrix: Embedding matrix for Keras Embedding layer
    """
    # First, build the "word2coefs" and "word2index"
    word2coefs = {} # word to its corresponding coefficients
    word2index = {} # word to word-index
    with open(fp) as f:
        for idx, line in enumerate(f):
            try:
                data = [x.strip().lower() for x in line.split()]
                word = data[0]
                coefs = np.asarray(data[1:embedding_dim+1], dtype='float32')
                word2coefs[word] = coefs
                if word not in word2index:
                    word2index[word] = len(word2index)
            except Exception as e:
                print('Exception occurred in `load_glove_embeddings`:', e)
                continue
        # End of for loop.
    # End of with open
    if include_empty_char:
        word2index[''] = len(word2index)
    # Second, build the "embedding_matrix"
    # Words not found in embedding index will be all-zeros. Hence, the "+1".
    vocab_size = len(word2coefs)+1 if include_empty_char else len(word2coefs)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, idx in word2index.items():
        embedding_vec = word2coefs.get(word)
        if embedding_vec is not None and embedding_vec.shape[0]==embedding_dim:
            embedding_matrix[idx] = np.asarray(embedding_vec)
    # return word2coefs, word2index, embedding_matrix
    return word2index, np.asarray(embedding_matrix)

[nltk_data] Downloading package punkt to /home/ahlem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Charger les embeddings à l'aide de la fonction load_glove_embeddings

word2index, embedding_matrix = load_glove_embeddings('glove.6B.50d.txt', embedding_dim=50)

In [12]:
word2index['unk']

201534

In [19]:
# ecrire une fonction de tokenization custom pour preprocesser les textes
unknown_words = set()
known_words = set()

def custom_tokenize(doc):
  res = []
  for ex in word_tokenize(doc): 
    if ex not in word2index.keys():
      res.append(word2index['unk'])
      unknown_words.add(ex)
    else:
      res.append(word2index[ex])
      known_words.add(ex)
  return np.array(res)
    
# Encoder les textes avec la fonction custom
X_train_glove = [custom_tokenize(x) for x in x_train]
print(x_train[0])
print(X_train_glove[0])

X_test_glove = [custom_tokenize(x) for x in x_test]
print(X_test_glove[0])


an improved kernel based extreme learning machine for robot execution failures.
[   29  2338 21566   243  3828  2741  2358    10  9247  4415  7693     2]
[ 6707 13867  2233     6   426  2022  2817     2]


In [22]:
print(f"With Glove, there is {len(unknown_words)} unknown words and {len(known_words)} known words.")

With Glove, there is 4592 unknown words and 11324 known words.


In [8]:
# Padding des sequences

X_train_glove = tf.keras.preprocessing.sequence.pad_sequences(X_train_glove, maxlen = 40, truncating='post')
X_test_glove = tf.keras.preprocessing.sequence.pad_sequences(X_test_glove, maxlen = 40, truncating='post')

# CNN
## Modèle basique
## Embedding non préentrainés

In [10]:
embed_dim = 128
batch_size = 128

model = tf.keras.models.Sequential() # modèle séquentiel
model.add(tf.keras.layers.Embedding(10000, embed_dim,input_length = 40))# couche d'embedding de taille 128
model.add(tf.keras.layers.Conv1D(32,
                 2,
                 padding='valid',
                 activation='relu',
                 strides=1))

model.add(tf.keras.layers.GlobalMaxPooling1D())

model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dense(cat_vocab))
model.add(tf.keras.layers.Activation('softmax'))

# Compiler le modèle 

model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

# Afficher le summary du modèle
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 128)           1280000   
_________________________________________________________________
conv1d (Conv1D)              (None, 39, 32)            8224      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               4224      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 93)                11997     
__________

In [11]:
# Fitter le modèle 

history = model.fit(x_train_seq, y_train_id, batch_size = batch_size, epochs = 10, validation_data=(x_test_seq, y_test_id))

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 2s 263us/step - loss: 3.9159 - acc: 0.1690 - val_loss: 3.4554 - val_acc: 0.1880
Epoch 2/10
8000/8000 [==============================] - 0s 19us/step - loss: 3.3196 - acc: 0.2320 - val_loss: 3.1856 - val_acc: 0.2675
Epoch 3/10
8000/8000 [==============================] - 0s 19us/step - loss: 3.0286 - acc: 0.2970 - val_loss: 2.9929 - val_acc: 0.3130
Epoch 4/10
8000/8000 [==============================] - 0s 18us/step - loss: 2.7075 - acc: 0.3621 - val_loss: 2.8633 - val_acc: 0.3310
Epoch 5/10
8000/8000 [==============================] - 0s 18us/step - loss: 2.3373 - acc: 0.4282 - val_loss: 2.7857 - val_acc: 0.3510
Epoch 6/10
8000/8000 [==============================] - 0s 19us/step - loss: 1.9620 - acc: 0.5084 - val_loss: 2.8051 - val_acc: 0.3535
Epoch 7/10
8000/8000 [==============================] - 0s 19us/step - loss: 1.6304 - acc: 0.5867 - val_loss: 2.9213 - val_acc: 0.3315
Epoch 

In [ ]:
#visualiser
plot_history(history)

In [12]:
# Evaluer le modèle
model.evaluate(x_test_seq, y_test_id)

2000/2000 [==============================] - 0s 75us/step


[3.485855577468872, 0.3205]

# CNN
## Modèle basique
## Embedding GloVE

In [15]:
# Entrainer un modèle en chargeant les poids des embeddings dans le layer Embedding

#embed_dim = 50

batch_size = 128

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],input_length = 40,
                                    weights=[embedding_matrix], 
                              trainable=True))
model.add(tf.keras.layers.Conv1D(32,
                 2,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(tf.keras.layers.GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dense(cat_vocab))
model.add(tf.keras.layers.Activation('softmax'))

adam = tf.keras.optimizers.Adam(lr=1e-3)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=adam, metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 50)            20000050  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 39, 32)            3232      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               4224      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
activation_4 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 93)                11997     
__________

In [16]:
history = model.fit(X_train_glove, y_train_id, batch_size = batch_size, epochs = 10, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 172us/step - loss: 3.8672 - acc: 0.1354 - val_loss: 3.4002 - val_acc: 0.2155
Epoch 2/10
8000/8000 [==============================] - 1s 72us/step - loss: 3.3837 - acc: 0.2245 - val_loss: 3.1642 - val_acc: 0.2650
Epoch 3/10
8000/8000 [==============================] - 1s 76us/step - loss: 3.1539 - acc: 0.2605 - val_loss: 3.0185 - val_acc: 0.2725
Epoch 4/10
8000/8000 [==============================] - 1s 77us/step - loss: 2.9960 - acc: 0.2899 - val_loss: 2.9066 - val_acc: 0.2905
Epoch 5/10
8000/8000 [==============================] - 1s 76us/step - loss: 2.8418 - acc: 0.3086 - val_loss: 2.7957 - val_acc: 0.3055
Epoch 6/10
8000/8000 [==============================] - 1s 72us/step - loss: 2.7185 - acc: 0.3284 - val_loss: 2.7085 - val_acc: 0.3145
Epoch 7/10
8000/8000 [==============================] - 1s 73us/step - loss: 2.5868 - acc: 0.3520 - val_loss: 2.6429 - val_acc: 0.3290
Epoch 

In [17]:
# Evaluer le modèle
model.evaluate(X_test_glove, y_test_id)

2000/2000 [==============================] - 0s 76us/step


[2.541342189788818, 0.34]

# CNN
## Modèle basique, Changement hyperparamètres
## Embedding non préentrainés

In [22]:
embed_dim = 128
batch_size = 128

model = tf.keras.models.Sequential() # modèle séquentiel
model.add(tf.keras.layers.Embedding(10000, embed_dim,input_length = 40))# couche d'embedding de taille 
model.add(tf.keras.layers.Conv1D(256,
                 2,
                 padding='valid',
                 activation='relu',
                 strides=1))

model.add(tf.keras.layers.GlobalMaxPooling1D())

model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(cat_vocab))
model.add(tf.keras.layers.Activation('softmax'))

# Compiler le modèle 
adam = tf.keras.optimizers.Adam(lr=1e-4)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=adam ,metrics = ['accuracy'])
# Fitter le modèle 
history = model.fit(x_train_seq, y_train_id, batch_size = batch_size, epochs = 50, validation_data=(x_test_seq, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 1s 103us/step - loss: 4.4543 - acc: 0.1118 - val_loss: 4.3578 - val_acc: 0.1880
Epoch 2/50
8000/8000 [==============================] - 0s 24us/step - loss: 4.1809 - acc: 0.1878 - val_loss: 3.9181 - val_acc: 0.1880
Epoch 3/50
8000/8000 [==============================] - 0s 23us/step - loss: 3.7160 - acc: 0.1876 - val_loss: 3.5543 - val_acc: 0.1880
Epoch 4/50
8000/8000 [==============================] - 0s 23us/step - loss: 3.5791 - acc: 0.1870 - val_loss: 3.5007 - val_acc: 0.1880
Epoch 5/50
8000/8000 [==============================] - 0s 24us/step - loss: 3.5295 - acc: 0.1871 - val_loss: 3.4690 - val_acc: 0.1880
Epoch 6/50
8000/8000 [==============================] - 0s 25us/step - loss: 3.5086 - acc: 0.1876 - val_loss: 3.4440 - val_acc: 0.1885
Epoch 7/50
8000/8000 [==============================] - 0s 26us/step - loss: 3.4657 - acc: 0.2001 - val_loss: 3.4049 - val_acc: 0.2175
Epoch 

In [23]:
# Evaluer le modèle
model.evaluate(x_test_seq, y_test_id)

2000/2000 [==============================] - 0s 34us/step


[2.938798152923584, 0.3705]

# CNN
## Modèle basique, Changement hyperparamètres
## Embedding GloVe

In [24]:
#embed_dim = 50
batch_size = 128

model = tf.keras.models.Sequential() # modèle séquentiel
model.add(tf.keras.layers.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],input_length = 40,
                                    weights=[embedding_matrix], 
                              trainable=True))# couche d'embedding de taille 
model.add(tf.keras.layers.Conv1D(256,
                 2,
                 padding='valid',
                 activation='relu',
                 strides=1))


model.add(tf.keras.layers.GlobalMaxPooling1D())

model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Activation('relu'))

model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(cat_vocab))
model.add(tf.keras.layers.Activation('softmax'))

# Compiler le modèle 
adam = tf.keras.optimizers.Adam(lr=1e-4)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=adam ,metrics = ['accuracy'])
# Fitter le modèle 
history = model.fit(X_train_glove, y_train_id, batch_size = batch_size, epochs = 50, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 2s 218us/step - loss: 4.1738 - acc: 0.1159 - val_loss: 3.7418 - val_acc: 0.1880
Epoch 2/50
8000/8000 [==============================] - 1s 75us/step - loss: 3.7465 - acc: 0.1796 - val_loss: 3.5511 - val_acc: 0.1880
Epoch 3/50
8000/8000 [==============================] - 1s 81us/step - loss: 3.6235 - acc: 0.1878 - val_loss: 3.4553 - val_acc: 0.1930
Epoch 4/50
8000/8000 [==============================] - 1s 83us/step - loss: 3.5207 - acc: 0.2048 - val_loss: 3.3781 - val_acc: 0.2200
Epoch 5/50
8000/8000 [==============================] - 1s 77us/step - loss: 3.4455 - acc: 0.2174 - val_loss: 3.3054 - val_acc: 0.2445
Epoch 6/50
8000/8000 [==============================] - 1s 79us/step - loss: 3.3538 - acc: 0.2324 - val_loss: 3.2388 - val_acc: 0.2610
Epoch 7/50
8000/8000 [==============================] - 1s 83us/step - loss: 3.2778 - acc: 0.2496 - val_loss: 3.1778 - val_acc: 0.2665
Epoch 

In [26]:
# Evaluer le modèle
model.evaluate(X_test_glove, y_test_id)

2000/2000 [==============================] - 0s 90us/step


[2.3543439178466796, 0.377]

# CNN
## Modèle CNN multi-channel
## Embedding non préentrainés

In [9]:
def get_cnn_model():
    embedding_dim = 300
    filter_sizes = [2, 3, 5]
    num_filters = 256
    drop = 0.3
    MAX_LENGTH=40
    MAX_NB_WORDS = 10000
    
    inputs = Input(shape=(MAX_LENGTH,), dtype='int32')
    embedding = Embedding(input_dim=MAX_NB_WORDS,
                                output_dim=embedding_dim,
                                input_length=MAX_LENGTH)(inputs)

    reshape = Reshape((MAX_LENGTH, embedding_dim, 1))(embedding)
    conv_0 = Conv2D(num_filters, 
                    kernel_size=(filter_sizes[0], embedding_dim), 
                    padding='valid', kernel_initializer='normal', 
                    activation='relu')(reshape)

    conv_1 = Conv2D(num_filters, 
                    kernel_size=(filter_sizes[1], embedding_dim), 
                    padding='valid', kernel_initializer='normal', 
                    activation='relu')(reshape)
    conv_2 = Conv2D(num_filters, 
                    kernel_size=(filter_sizes[2], embedding_dim), 
                    padding='valid', kernel_initializer='normal', 
                    activation='relu')(reshape)

    maxpool_0 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[0] + 1, 1), 
                          strides=(1,1), padding='valid')(conv_0)

    maxpool_1 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[1] + 1, 1), 
                          strides=(1,1), padding='valid')(conv_1)

    maxpool_2 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[2] + 1, 1), 
                          strides=(1,1), padding='valid')(conv_2)
    concatenated_tensor = Concatenate(axis=1)(
        [maxpool_0, maxpool_1, maxpool_2])
    flatten = Flatten()(concatenated_tensor)
    
    dropout = Dropout(drop)(flatten)
    
    #outpout1=Dense(units=256, activation='relu')(dropout)
    #dropout1 = Dropout(drop)(outpout1)
    
    output = Dense(units=cat_vocab, activation='softmax')(dropout)

    model = Model(inputs=inputs, outputs=output)
    
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

cnn_model_multi_channel = get_cnn_model()

batch_size = 128
epochs = 45
history = cnn_model_multi_channel.fit(x=x_train_seq, 
                    y=y_train_id, 
                    batch_size=batch_size, 
                    validation_data=(x_test_seq, y_test_id),
                    epochs=epochs)

Train on 8000 samples, validate on 2000 samples
Epoch 1/45
8000/8000 [==============================] - 3s 386us/step - loss: 4.2620 - acc: 0.1373 - val_loss: 3.9116 - val_acc: 0.1880
Epoch 2/45
8000/8000 [==============================] - 1s 116us/step - loss: 3.6128 - acc: 0.1894 - val_loss: 3.4749 - val_acc: 0.1880
Epoch 3/45
8000/8000 [==============================] - 1s 119us/step - loss: 3.4284 - acc: 0.1933 - val_loss: 3.4192 - val_acc: 0.1955
Epoch 4/45
8000/8000 [==============================] - 1s 113us/step - loss: 3.3641 - acc: 0.2151 - val_loss: 3.3702 - val_acc: 0.2205
Epoch 5/45
8000/8000 [==============================] - 1s 115us/step - loss: 3.2994 - acc: 0.2333 - val_loss: 3.3141 - val_acc: 0.2360
Epoch 6/45
8000/8000 [==============================] - 1s 129us/step - loss: 3.2293 - acc: 0.2449 - val_loss: 3.2620 - val_acc: 0.2515
Epoch 7/45
8000/8000 [==============================] - 1s 139us/step - loss: 3.1709 - acc: 0.2509 - val_loss: 3.2208 - val_acc: 0.2520


In [10]:
# Evaluer le modèle
cnn_model_multi_channel.evaluate(x_test_seq, y_test_id)

2000/2000 [==============================] - 0s 133us/step


[2.5053614768981936, 0.403]

In [ ]:
plot_history(history)

# CNN
## Modèle CNN multi-channel
## Embedding GloVe

In [9]:
embedding_matrix.shape

(400001, 50)

In [11]:
def get_cnn_model():
    embedding_dim = 50
    filter_sizes = [2, 3, 5]
    num_filters = 256
    drop = 0.3
    MAX_LENGTH=40
    MAX_NB_WORDS = 10000
    
    inputs = Input(shape=(MAX_LENGTH,), dtype='int32')
    embedding = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],input_length = 40,
                                    weights=[embedding_matrix], 
                              trainable=True)(inputs)

    reshape = Reshape((MAX_LENGTH, embedding_dim, 1))(embedding)
    conv_0 = Conv2D(num_filters, 
                    kernel_size=(filter_sizes[0], embedding_dim), 
                    padding='valid', kernel_initializer='normal', 
                    activation='relu')(reshape)

    conv_1 = Conv2D(num_filters, 
                    kernel_size=(filter_sizes[1], embedding_dim), 
                    padding='valid', kernel_initializer='normal', 
                    activation='relu')(reshape)
    conv_2 = Conv2D(num_filters, 
                    kernel_size=(filter_sizes[2], embedding_dim), 
                    padding='valid', kernel_initializer='normal', 
                    activation='relu')(reshape)

    maxpool_0 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[0] + 1, 1), 
                          strides=(1,1), padding='valid')(conv_0)

    maxpool_1 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[1] + 1, 1), 
                          strides=(1,1), padding='valid')(conv_1)

    maxpool_2 = MaxPool2D(pool_size=(MAX_LENGTH - filter_sizes[2] + 1, 1), 
                          strides=(1,1), padding='valid')(conv_2)
    concatenated_tensor = Concatenate(axis=1)(
        [maxpool_0, maxpool_1, maxpool_2])
    flatten = Flatten()(concatenated_tensor)
    
    dropout = Dropout(drop)(flatten)
    
    #outpout1=Dense(units=256, activation='relu')(dropout)
    #dropout1 = Dropout(drop)(outpout1)
    
    output = Dense(units=cat_vocab, activation='softmax')(dropout)

    model = Model(inputs=inputs, outputs=output)
    
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = get_cnn_model()
batch_size = 128

In [12]:
history = model.fit(X_train_glove, y_train_id, batch_size = batch_size, epochs = 45, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/45
8000/8000 [==============================] - 2s 254us/step - loss: 4.0113 - acc: 0.1211 - val_loss: 3.4170 - val_acc: 0.2065
Epoch 2/45
8000/8000 [==============================] - 1s 119us/step - loss: 3.4965 - acc: 0.1895 - val_loss: 3.2945 - val_acc: 0.2355
Epoch 3/45
8000/8000 [==============================] - 1s 119us/step - loss: 3.3612 - acc: 0.2209 - val_loss: 3.2022 - val_acc: 0.2700
Epoch 4/45
8000/8000 [==============================] - 1s 120us/step - loss: 3.2472 - acc: 0.2501 - val_loss: 3.1302 - val_acc: 0.2825
Epoch 5/45
8000/8000 [==============================] - 1s 119us/step - loss: 3.1409 - acc: 0.2699 - val_loss: 3.0645 - val_acc: 0.2895
Epoch 6/45
8000/8000 [==============================] - 1s 121us/step - loss: 3.0620 - acc: 0.2775 - val_loss: 3.0107 - val_acc: 0.2945
Epoch 7/45
8000/8000 [==============================] - 1s 120us/step - loss: 2.9967 - acc: 0.2923 - val_loss: 2.9612 - val_acc: 0.3035


In [14]:
history = model.fit(X_train_glove, y_train_id, batch_size = batch_size, epochs = 45, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/45
8000/8000 [==============================] - 1s 135us/step - loss: 1.7136 - acc: 0.5439 - val_loss: 2.3452 - val_acc: 0.3905
Epoch 2/45
8000/8000 [==============================] - 1s 120us/step - loss: 1.6896 - acc: 0.5390 - val_loss: 2.3411 - val_acc: 0.3865
Epoch 3/45
8000/8000 [==============================] - 1s 120us/step - loss: 1.6670 - acc: 0.5487 - val_loss: 2.3364 - val_acc: 0.3860
Epoch 4/45
8000/8000 [==============================] - 1s 122us/step - loss: 1.6531 - acc: 0.5541 - val_loss: 2.3324 - val_acc: 0.3900
Epoch 5/45
8000/8000 [==============================] - 1s 123us/step - loss: 1.6312 - acc: 0.5646 - val_loss: 2.3274 - val_acc: 0.3855
Epoch 6/45
8000/8000 [==============================] - 1s 119us/step - loss: 1.6191 - acc: 0.5619 - val_loss: 2.3241 - val_acc: 0.3860
Epoch 7/45
8000/8000 [==============================] - 1s 120us/step - loss: 1.5830 - acc: 0.5705 - val_loss: 2.3205 - val_acc: 0.3890


In [15]:
# Evaluer le modèle
model.evaluate(X_test_glove, y_test_id)

2000/2000 [==============================] - 0s 138us/step


[2.2661995372772217, 0.402]

# RNN
## Modèle basique: LSTM
## Embedding non préentrainés

In [18]:
# Créer un réseau à base de LSTM avec au minimum:
# Embedding
# Dropout
# LSTM
# Dropout
# Classifieur

embed_dim = 128
lstm_out = 128
batch_size = 128

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(10000, embed_dim,input_length = 40))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_out)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(cat_vocab,activation='softmax'))

# Compiler le modèle 
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

# Afficher le summary du modèle
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 128)           1280000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 40, 128)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 93)                11997     
Total para

In [19]:
# Fitter le modèle 

history = model.fit(x_train_seq, y_train_id, batch_size = batch_size, epochs = 5, validation_data=(x_test_seq, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 6s 746us/step - loss: 3.7560 - acc: 0.1670 - val_loss: 3.4126 - val_acc: 0.2315
Epoch 2/5
8000/8000 [==============================] - 5s 565us/step - loss: 3.2691 - acc: 0.2446 - val_loss: 3.1396 - val_acc: 0.2560
Epoch 3/5
8000/8000 [==============================] - 4s 562us/step - loss: 2.9673 - acc: 0.2836 - val_loss: 2.9465 - val_acc: 0.3000
Epoch 4/5
8000/8000 [==============================] - 4s 562us/step - loss: 2.6344 - acc: 0.3385 - val_loss: 2.8653 - val_acc: 0.3275
Epoch 5/5
8000/8000 [==============================] - 4s 561us/step - loss: 2.2948 - acc: 0.4183 - val_loss: 2.8664 - val_acc: 0.3265


In [20]:
# Evaluer le modèle
model.evaluate(x_test_seq, y_test_id)

2000/2000 [==============================] - 2s 801us/step


[2.866437623977661, 0.3265]

# RNN
## Modèle basique: LSTM
## Embedding Glove

In [23]:

lstm_out = 128
batch_size = 128

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],input_length = 40,
                                    weights=[embedding_matrix], 
                              trainable=True))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_out)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(cat_vocab,activation='softmax'))

# Compiler le modèle 
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

# Afficher le summary du modèle
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 40, 50)            20000050  
_________________________________________________________________
dropout_9 (Dropout)          (None, 40, 50)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               183296    
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 93)                11997     
Total para

In [24]:
history = model.fit(X_train_glove, y_train_id, batch_size = batch_size, epochs = 18, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/18
8000/8000 [==============================] - 7s 888us/step - loss: 3.7192 - acc: 0.1717 - val_loss: 3.3189 - val_acc: 0.2345
Epoch 2/18
8000/8000 [==============================] - 5s 615us/step - loss: 3.2145 - acc: 0.2476 - val_loss: 2.9274 - val_acc: 0.2880
Epoch 3/18
8000/8000 [==============================] - 5s 620us/step - loss: 2.9693 - acc: 0.2834 - val_loss: 2.7681 - val_acc: 0.3140
Epoch 4/18
8000/8000 [==============================] - 5s 614us/step - loss: 2.8094 - acc: 0.3040 - val_loss: 2.6647 - val_acc: 0.3315
Epoch 5/18
8000/8000 [==============================] - 5s 617us/step - loss: 2.6754 - acc: 0.3250 - val_loss: 2.5819 - val_acc: 0.3505
Epoch 6/18
8000/8000 [==============================] - 5s 616us/step - loss: 2.5728 - acc: 0.3450 - val_loss: 2.5269 - val_acc: 0.3450
Epoch 7/18
8000/8000 [==============================] - 5s 620us/step - loss: 2.4876 - acc: 0.3568 - val_loss: 2.4924 - val_acc: 0.3520


In [25]:
# Evaluer le modèle
model.evaluate(X_test_glove, y_test_id)

2000/2000 [==============================] - 2s 775us/step


[2.340456693649292, 0.399]

# RNN
## Modèle basique: GRU
## Embedding non préentrainés

In [28]:
def get_simple_rnn_model():
    embedding_dim = 300
    MAX_NB_WORDS = 10000
    MAX_LENGTH=40
    embedding_matrix = np.random.random((MAX_NB_WORDS, embedding_dim))
    
    inp = Input(shape=(MAX_LENGTH, ))
    x = Embedding(input_dim=MAX_NB_WORDS, output_dim=embedding_dim, input_length=MAX_LENGTH, 
                  weights=[embedding_matrix], trainable=True)(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(cat_vocab, activation="softmax")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    adam = Adam(lr=1e-4)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer= adam,
                  metrics=['accuracy'])
    return model

rnn_simple_model = get_simple_rnn_model()

In [29]:
batch_size = 128
epochs = 150
history = rnn_simple_model .fit(x=x_train_seq, 
                    y=y_train_id, 
                    batch_size=batch_size, 
                    validation_data=(x_test_seq, y_test_id),
                    epochs=epochs)

Train on 8000 samples, validate on 2000 samples
Epoch 1/150
8000/8000 [==============================] - 5s 652us/step - loss: 3.9082 - acc: 0.1584 - val_loss: 3.5130 - val_acc: 0.1880
Epoch 2/150
8000/8000 [==============================] - 3s 418us/step - loss: 3.5118 - acc: 0.1878 - val_loss: 3.4665 - val_acc: 0.1880
Epoch 3/150
8000/8000 [==============================] - 3s 413us/step - loss: 3.4816 - acc: 0.1884 - val_loss: 3.4550 - val_acc: 0.1880
Epoch 4/150
8000/8000 [==============================] - 3s 413us/step - loss: 3.4711 - acc: 0.1889 - val_loss: 3.4519 - val_acc: 0.1880
Epoch 5/150
8000/8000 [==============================] - 3s 415us/step - loss: 3.4593 - acc: 0.1885 - val_loss: 3.4439 - val_acc: 0.1880
Epoch 6/150
8000/8000 [==============================] - 3s 411us/step - loss: 3.4560 - acc: 0.1890 - val_loss: 3.4391 - val_acc: 0.1880
Epoch 7/150
8000/8000 [==============================] - 3s 412us/step - loss: 3.4547 - acc: 0.1898 - val_loss: 3.4328 - val_acc: 

8000/8000 [==============================] - 3s 412us/step - loss: 2.5991 - acc: 0.3709 - val_loss: 2.7874 - val_acc: 0.3375
Epoch 61/150
8000/8000 [==============================] - 3s 414us/step - loss: 2.5809 - acc: 0.3755 - val_loss: 2.7840 - val_acc: 0.3390
Epoch 62/150
8000/8000 [==============================] - 3s 416us/step - loss: 2.5666 - acc: 0.3777 - val_loss: 2.7730 - val_acc: 0.3400
Epoch 63/150
8000/8000 [==============================] - 3s 414us/step - loss: 2.5468 - acc: 0.3814 - val_loss: 2.7626 - val_acc: 0.3475
Epoch 64/150
8000/8000 [==============================] - 3s 413us/step - loss: 2.5322 - acc: 0.3865 - val_loss: 2.7555 - val_acc: 0.3510
Epoch 65/150
8000/8000 [==============================] - 3s 412us/step - loss: 2.5249 - acc: 0.3890 - val_loss: 2.7516 - val_acc: 0.3485
Epoch 66/150
8000/8000 [==============================] - 3s 415us/step - loss: 2.5109 - acc: 0.3891 - val_loss: 2.7427 - val_acc: 0.3520
Epoch 67/150
8000/8000 [=======================

In [30]:
rnn_simple_model.evaluate(x_test_seq, y_test_id)

2000/2000 [==============================] - 1s 546us/step


[2.580115997314453, 0.38]

# RNN
## Modèle basique: GRU
## Embedding Glove

In [31]:
def get_simple_rnn_model():
    embedding_dim = 50
    MAX_NB_WORDS = 10000
    MAX_LENGTH=40
    
    inp = Input(shape=(MAX_LENGTH, ))
    x = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],input_length = 40,
                                    weights=[embedding_matrix], 
                              trainable=True)(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(cat_vocab, activation="softmax")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    adam = Adam(lr=1e-4)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer= adam,
                  metrics=['accuracy'])
    return model

model = get_simple_rnn_model()

In [32]:
history = model.fit(X_train_glove, y_train_id, batch_size = 128, epochs = 150, validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/150
8000/8000 [==============================] - 6s 747us/step - loss: 4.2791 - acc: 0.0546 - val_loss: 3.8614 - val_acc: 0.1860
Epoch 2/150
8000/8000 [==============================] - 4s 460us/step - loss: 3.6758 - acc: 0.1842 - val_loss: 3.5093 - val_acc: 0.1880
Epoch 3/150
8000/8000 [==============================] - 4s 468us/step - loss: 3.4969 - acc: 0.1885 - val_loss: 3.4375 - val_acc: 0.1880
Epoch 4/150
8000/8000 [==============================] - 4s 459us/step - loss: 3.4434 - acc: 0.1891 - val_loss: 3.4048 - val_acc: 0.1880
Epoch 5/150
8000/8000 [==============================] - 4s 461us/step - loss: 3.4110 - acc: 0.1913 - val_loss: 3.3774 - val_acc: 0.1885
Epoch 6/150
8000/8000 [==============================] - 4s 458us/step - loss: 3.3834 - acc: 0.2003 - val_loss: 3.3520 - val_acc: 0.1940
Epoch 7/150
8000/8000 [==============================] - 4s 456us/step - loss: 3.3538 - acc: 0.2082 - val_loss: 3.3215 - val_acc: 

8000/8000 [==============================] - 4s 456us/step - loss: 2.3208 - acc: 0.3869 - val_loss: 2.4127 - val_acc: 0.3635
Epoch 61/150
8000/8000 [==============================] - 4s 460us/step - loss: 2.3192 - acc: 0.3904 - val_loss: 2.4136 - val_acc: 0.3725
Epoch 62/150
8000/8000 [==============================] - 4s 460us/step - loss: 2.3028 - acc: 0.3885 - val_loss: 2.4037 - val_acc: 0.3725
Epoch 63/150
8000/8000 [==============================] - 4s 458us/step - loss: 2.2983 - acc: 0.3824 - val_loss: 2.4019 - val_acc: 0.3715
Epoch 64/150
8000/8000 [==============================] - 4s 457us/step - loss: 2.2951 - acc: 0.3865 - val_loss: 2.3977 - val_acc: 0.3700
Epoch 65/150
8000/8000 [==============================] - 4s 458us/step - loss: 2.2806 - acc: 0.3926 - val_loss: 2.3841 - val_acc: 0.3755
Epoch 66/150
8000/8000 [==============================] - 4s 458us/step - loss: 2.2764 - acc: 0.3903 - val_loss: 2.3886 - val_acc: 0.3720
Epoch 67/150
8000/8000 [=======================

In [33]:
model.evaluate(X_test_glove, y_test_id)

2000/2000 [==============================] - 1s 573us/step


[2.2582788410186767, 0.395]

# CNN_RNN

## Embedding non préentrainés

In [34]:
def get_rnn_cnn_model():
    embedding_dim = 300
    MAX_NB_WORDS = 10000
    MAX_LENGTH=40
    embedding_matrix = np.random.random((MAX_NB_WORDS, embedding_dim))
    inp = Input(shape=(40, ))
    x = Embedding(MAX_NB_WORDS, embedding_dim, weights=[embedding_matrix], input_length=MAX_LENGTH, trainable=True)(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(cat_vocab, activation="softmax")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

model = get_rnn_cnn_model()
batch_size = 128
epochs = 15
history = model.fit(x=x_train_seq, 
                    y=y_train_id, 
                    batch_size=batch_size, 
                    validation_data=(x_test_seq, y_test_id),
                    epochs=epochs)

Train on 8000 samples, validate on 2000 samples
Epoch 1/15
8000/8000 [==============================] - 6s 722us/step - loss: 3.6181 - acc: 0.1758 - val_loss: 3.4511 - val_acc: 0.1900
Epoch 2/15
8000/8000 [==============================] - 3s 426us/step - loss: 3.4283 - acc: 0.2122 - val_loss: 3.3014 - val_acc: 0.2505
Epoch 3/15
8000/8000 [==============================] - 3s 422us/step - loss: 3.2694 - acc: 0.2461 - val_loss: 3.1884 - val_acc: 0.2705
Epoch 4/15
8000/8000 [==============================] - 3s 417us/step - loss: 3.1529 - acc: 0.2654 - val_loss: 3.1170 - val_acc: 0.2910
Epoch 5/15
8000/8000 [==============================] - 3s 417us/step - loss: 3.0234 - acc: 0.2901 - val_loss: 3.0412 - val_acc: 0.3085
Epoch 6/15
8000/8000 [==============================] - 3s 419us/step - loss: 2.9029 - acc: 0.3130 - val_loss: 2.9477 - val_acc: 0.3175
Epoch 7/15
8000/8000 [==============================] - 3s 417us/step - loss: 2.7715 - acc: 0.3371 - val_loss: 2.8694 - val_acc: 0.3325


In [35]:
model.evaluate(x_test_seq, y_test_id)

2000/2000 [==============================] - 1s 610us/step


[2.803877523422241, 0.349]

# CNN_RNN

## Embedding GloVe

In [36]:
def get_rnn_cnn_model():
    embedding_dim = 50
    MAX_NB_WORDS = 10000
    MAX_LENGTH=40
    
    inp = Input(shape=(40, ))
    x = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],input_length = 40,
                                    weights=[embedding_matrix], 
                              trainable=True)(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(cat_vocab, activation="softmax")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

model = get_rnn_cnn_model()
batch_size = 128
epochs = 15
history = model.fit(X_train_glove,
                    y_train_id, 
                    batch_size = batch_size,
                    epochs = epochs,
                    validation_data=(X_test_glove, y_test_id))

Train on 8000 samples, validate on 2000 samples
Epoch 1/15
8000/8000 [==============================] - 7s 813us/step - loss: 3.5522 - acc: 0.1958 - val_loss: 3.1862 - val_acc: 0.2730
Epoch 2/15
8000/8000 [==============================] - 4s 459us/step - loss: 3.0713 - acc: 0.2751 - val_loss: 2.8666 - val_acc: 0.2995
Epoch 3/15
8000/8000 [==============================] - 4s 465us/step - loss: 2.8125 - acc: 0.3050 - val_loss: 2.6860 - val_acc: 0.3235
Epoch 4/15
8000/8000 [==============================] - 4s 461us/step - loss: 2.6535 - acc: 0.3314 - val_loss: 2.5593 - val_acc: 0.3490
Epoch 5/15
8000/8000 [==============================] - 4s 463us/step - loss: 2.5127 - acc: 0.3616 - val_loss: 2.4746 - val_acc: 0.3570
Epoch 6/15
8000/8000 [==============================] - 4s 469us/step - loss: 2.3971 - acc: 0.3819 - val_loss: 2.4147 - val_acc: 0.3665
Epoch 7/15
8000/8000 [==============================] - 4s 460us/step - loss: 2.3084 - acc: 0.3939 - val_loss: 2.4008 - val_acc: 0.3635


In [37]:
model.evaluate(X_test_glove, y_test_id)

2000/2000 [==============================] - 1s 549us/step


[2.3368661251068117, 0.3885]